# \*\*\[ANOMALOUS\]\*\*Graph

SEOYEON CHOI  
2023-09-07

# Reference

[공식홈](https://docs.pygod.org/en/latest/generated/pygod.detector.ANOMALOUS.html)

[paper](https://www.ijcai.org/Proceedings/2018/0488.pdf)

[교수님께서 알려주신
사이트](https://pycaret.gitbook.io/docs/get-started/quickstart#anomaly-detection)

`Summary`

-   노드당 매핑된 attribute로 이상치를 계산해낸다.
-   그래서 attribute 특징마다 나오는 이상치라고 칭하는 노드가 다른 것
    같다.
-   노드 정보와 네트워크를 기반으로 rare하거나 상당히 differ한 인스턴스
    집합 찾는 것을 목표로 한다.

[데이터셋 논문](https://arxiv.org/pdf/1603.08861.pdf)

# Import

In [1]:
import pygod
import numpy as np
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid

import torch
from pygod.generator import gen_contextual_outlier, gen_structural_outlier

from pygod.utils import load_data

from pygod.metric import eval_roc_auc

from pygod.detector import ANOMALOUS

/home/csy/anaconda3/envs/temp_csy/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

# Tutorial

## CiteSeer Dataset

In [68]:
data = Planetoid('./data/CiteSeer', 'CiteSeer', transform=T.NormalizeFeatures())[0]

In [69]:
data

`gen_contextual_outlier`의 역할: Generating contextual outliers

-   임의로 선택한 노드 중 그 노드들끼리 얼마나 떨어져 있나?

In [70]:
data, ya = gen_contextual_outlier(data, n=100, k=50)

In [71]:
ya

In [72]:
len(sum(np.where(ya==1)))

In [73]:
len(sum(np.where(ya==0)))

In [74]:
len(ya)

`gen_structural_outlier`의 역할: Generating structural outliers

-   임의로 선택한 노드들이 fully connected 되어있을때 그 집단과 얼마나
    많이 다른가??

In [75]:
data, ys = gen_structural_outlier(data, m=10, n=10)

In [76]:
ys

In [77]:
len(sum(np.where(ys==1)))

In [78]:
len(sum(np.where(ys==0)))

In [79]:
len(ys)

위에서 찾은 이상치 간에 `torch.logical_or` 논리 or 생성

In [80]:
data.y = torch.logical_or(ys, ya).long()

In [81]:
data.y

In [82]:
len(sum(np.where(data.y==1)))

In [83]:
len(sum(np.where(data.y==0)))

In [84]:
len(data.y)

In [85]:
data = load_data('inj_cora')
data.y = data.y.bool()

In [86]:
data.y

`self.bool` 로 참인지 거짓인지 가릴 수 있다.

-   여기서는 참이면 이상치인듯

‘ANOMALOUS’ 함수 사용

In [87]:
detector = ANOMALOUS(gamma=1.,
                     weight_decay=0.,
                     lr=0.01,
                     epoch=50,
                     gpu=-1,
                     contamination=0.1,
                     verbose=0)

In [88]:
detector.fit(data)

``` python
class ANOMALOUSBase(nn.Module):
    def __init__(self, w, r):
        super(ANOMALOUSBase, self).__init__()
        self.w = nn.Parameter(w)
        self.r = nn.Parameter(r)

    def forward(self, x):
        return x @ self.w @ x, self.r
```

In [89]:
detector.decision_function(data)

/home/csy/anaconda3/envs/temp_csy/lib/python3.8/site-packages/pygod/detector/anomalous.py:111: UserWarning: This detector is transductive only. Training from scratch with the input data.
  warnings.warn("This detector is transductive only. "

위에서 decision_function의 결과로 나오는 decision_score는 r의 제곱이며,
이 r은 model에서 나온 결과인데 이 model은 ANOMALOUSBase(w_init,
r_init)의 결과이다.

이 r_init은 ANOMALOUS class 내에 있는 x, s, l, w_init, r_init =
self.process_graph(data) 여기서 나온다.

`-` return되는 거는 순서대로 x, s, laplacian, w_init, r_init

`x`

In [90]:
detector.process_graph(data)[0]

In [91]:
detector.process_graph(data)[0].shape

$X \in \mathbb{R}^{d \times n}$

2708 = `n` = the number of nodes

1433 = `d` = dimensiotnalattribute

`s`

In [92]:
detector.process_graph(data)[1]

In [93]:
detector.process_graph(data)[1].shape

$A \in \mathbb{R}^{n \times n}$

`laplacian`

In [94]:
detector.process_graph(data)[2]

In [95]:
detector.process_graph(data)[2].shape

`generated Laplacian`

$\tilde{R} L \tilde{R}^T$

`w_init`

In [96]:
detector.process_graph(data)[3]

In [97]:
detector.process_graph(data)[3].shape

`r_init`

In [98]:
detector.process_graph(data)[4]

In [99]:
detector.process_graph(data)[4].shape